# Clustering

# K means Clustering

In [3]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [4]:
data = pd.read_csv("C:/Shiva/Projects/GS_fault_Text_analytics/Medallia/Data/2016-18 All Medallia Surveys Aug 31_2.csv")

In [5]:
data = data['Likelihood to Recommend Comment'].tolist()

In [6]:
print data[:2]

["Because in recent years with BT's cooperation, feel BT professional, it is recommended", 'I enjoy the interaction with the technical and the account teams.']


In [75]:
print data[0][:10] #first 200 characters in first synopses (for 'The Godfather')


# Stemming, stopwords and tokenizing

In [7]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')


In [8]:
print stopwords[:10]

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your']


In [9]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")


In [44]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens


In [45]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in data:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)


In [47]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame'


In [53]:
vocab_frame.head()

,words
becaus,because
in,in
recent,recent
year,years
with,with


In [49]:
type(vocab_frame)

pandas.core.frame.DataFrame

In [54]:
len(vocab_frame)

21594

In [ ]:
data = [i.encode('utf-8').strip() for i in data]

In [109]:
tfidf_matrix.shape

(887, 2)

In [106]:
type(tfidf_matrix)

scipy.sparse.csr.csr_matrix

In [107]:
tfidf_matrix

<887x2 sparse matrix of type '<type 'numpy.float64'>'
	with 740 stored elements in Compressed Sparse Row format>

In [66]:
terms = tfidf_vectorizer.get_feature_names()


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [89]:
for i in data:
    i.decode('utf-8', errors='replace').encode('utf-8')

In [87]:
type(data)

list

In [88]:
data[0]

"Because in recent years with BT's cooperation, feel BT professional, it is recommended"

In [94]:
tfidf_vectorizer = TfidfVectorizer()

In [117]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(data) #fit the vectorizer to synopses

#print(tfidf_matrix.shape)


In [118]:
terms = tfidf_vectorizer.get_feature_names()

In [119]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print
print


<function print>

In [120]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [126]:
km

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [131]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(tfidf_matrix)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=2, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [132]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print "Cluster %d:" % i,
    for ind in order_centroids[i, :10]:
        print ' %s' % terms[ind],
    print


SyntaxError: invalid syntax (<ipython-input-132-d5430fddb929>, line 5)